In [2]:
import pandas as pd

C:\Users\drenergydrink\AppData\Local\Temp\ipykernel_13112\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
movies=pd.read_csv("tmdb_5000_movies.csv")

In [4]:
credits=pd.read_csv("tmdb_5000_credits.csv")

In [5]:
new_movies=movies.merge(credits,on="title")

In [6]:
new_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
#genre
#id
#keyword
#overview
#title
#cast
#crew these are the only columns we need

In [8]:
new_movies=new_movies[['id','title','genres','keywords','overview','cast','crew']]
new_movies.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
new_movies.dropna(inplace=True)

In [10]:
import json

In [11]:
def formatgenre(text):
    genres=[]
    for i in json.loads(text):
        genres.append(i["name"])
    return genres

In [12]:
new_movies['genres']=new_movies['genres'].apply(formatgenre)

In [13]:
def formatkeywords(text):
    keywords=[]
    for i in json.loads(text):
        keywords.append(i["name"])
    return keywords

In [14]:
new_movies['keywords']=new_movies['keywords'].apply(formatkeywords)

In [15]:
def castformatter(text):
    cast=[]
    z=0
    for i in json.loads(text):
        if z==3:
            break
        z=z+1
        cast.append(i["name"])
    return cast

In [16]:
new_movies['cast']=new_movies['cast'].apply(castformatter)

In [17]:
def crewformatter(text):
    director=[]
    for i in json.loads(text):
        if i["job"]=="Director":
            director.append(i["name"])
            break
    return director

In [18]:
new_movies['crew']=new_movies['crew'].apply(crewformatter)

In [19]:
new_movies["keywords"][0]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [20]:
new_movies["overview"]=new_movies["overview"].apply(lambda x: x.split())
new_movies['crew']=new_movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
new_movies['genres']=new_movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
new_movies['cast']=new_movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
new_movies['keywords']=new_movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
new_movies["allkeywords"]=new_movies["overview"]+new_movies["keywords"]+new_movies["genres"]+new_movies["cast"]+new_movies["crew"]

In [21]:
movies=new_movies[["id","title","allkeywords"]]

In [22]:
movies["allkeywords"]=movies["allkeywords"].apply(lambda x:" ".join(x))

C:\Users\drenergydrink\AppData\Local\Temp\ipykernel_13112\2329735072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["allkeywords"]=movies["allkeywords"].apply(lambda x:" ".join(x))


In [23]:
movies.head()

,id,title,allkeywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [24]:
movies["allkeywords"]=movies["allkeywords"].apply(lambda x:x.lower())

C:\Users\drenergydrink\AppData\Local\Temp\ipykernel_13112\3099174844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["allkeywords"]=movies["allkeywords"].apply(lambda x:x.lower())


In [25]:
movies["allkeywords"][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d action adventure fantasy sciencefiction samworthington zoesaldana sigourneyweaver jamescameron'

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words="english")

In [27]:
import nltk
from nltk.stem.porter import PorterStemmer

In [28]:
ps=PorterStemmer()

In [29]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [30]:
movies["allkeywords"]=movies["allkeywords"].apply(stem)

C:\Users\drenergydrink\AppData\Local\Temp\ipykernel_13112\3054010685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["allkeywords"]=movies["allkeywords"].apply(stem)


In [31]:
vectors=cv.fit_transform(movies["allkeywords"]).toarray()

In [32]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [33]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
similarity=cosine_similarity(vectors)

In [36]:
def recommend(movie):
    movie_index=movies[movies["title"]==movie].index[0]
    distances=similarity[movie_index]
    top_5=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in top_5:
        print(movies.iloc[i[0]].title)

In [37]:
recommend("Spiderman")

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.
